# Neighborhood-based Collaborative Filtering
(by Tevfik Aytekin)

In the following we will implement item based collaborative filtering.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from scipy.sparse import csr_matrix
from collections import Counter
from sklearn.metrics import pairwise_distances
from operator import itemgetter
from tqdm.notebook import tqdm
import copy
import heapq
import sys, os
import pickle
import itertools
import operator


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Movielens ml-latest-small dataset

In [ ]:
with open('/content/drive/My Drive/datasets/ml-latest-small/README.txt', 'r') as f:
    print(f.read())

In [ ]:
ratings = pd.read_csv("/content/drive/My Drive/datasets/ml-latest-small/ratings.csv", sep=",")
#ratings =pd.read_csv("/content/drive/My Drive/datasets/ml-25m/ratings.csv")
print(ratings.shape)
ratings.head(10)

In [ ]:
links = pd.read_csv("/content/drive/My Drive/datasets/ml-latest-small/links.csv", sep=",")
print(links.shape)
links.head()

In [ ]:
movies = pd.read_csv("/content/drive/My Drive/datasets/ml-latest-small/movies.csv", sep=",")
print(movies.shape)
movies.head()

In [ ]:
tags = pd.read_csv("/content/drive/My Drive/datasets/ml-latest-small/tags.csv", sep=",")
print(tags.shape)
tags.head()

## Create User Item Rating Map
It might take some time but will be useful later.

In [ ]:
rating_map = {}
for i in range(len(ratings)):
    key = str(ratings.iloc[i,0]) + '_' +str(ratings.iloc[i,1])
    rating_map[key]=ratings.iloc[i,2]

In [ ]:
rating_map["1_101"]

In [ ]:
iterator = iter(rating_map.items())
for i in range(5):
    print(next(iterator))

## Create User Ratings Map
It might take some time but will be useful later.

In [ ]:
ratings.query("movieId == 2")

In [ ]:
# user_ratings_map[i] stores a tuple: a list of users who rated item i and a list of corresponding ratings
user_ratings_map = {}

items = ratings.movieId.unique()
for i in items:
    userids = ratings.query("movieId == @i").userId.array
    user_ratings = ratings.query("movieId == @i").rating.array
    user_ratings_map[i] = (userids,user_ratings)

In [ ]:
user_ratings_map[10]

## Rating Prediction

### Algorithm

Predict rating of user $u$ for item $i$
- Calculate the similarity of items that are rated by $u$ with $i$.
- Use these similarities to calculate a weighted average of the ratings.

Similarity between items i and j will be calculated using the ratings of i and j (no content information will be used). One can view these ratings as a vector of values as shown below and use different metrics such as Jaccard or cosine.

In [ ]:
df = pd.DataFrame([[1, "", 5, 3, ""],
                   [5, 3, "", 2, 4],
                   ["", 4, 2, "", 1],
                   [3, "", 4, 2, 3],
                   [4, 1, "", 2, 4],
                   [4, 1, 5, 3, ""],
                   ["", 4, 5, "", 1],
                   [2, 5, "", 1, 4]],
                 index = ['user 1','user 2','user 3','user 4','user 5','user 6','user 7','user 8'],
                 columns = ['movie 1','movie 2','movie 3','movie 4',' movie 5'])
df

### Jaccard Similarity

Given two sets $A$ and $B$,

$Jaccard(A, B) = \frac{|A \cap B|}{|A \cup B|}$

For example if $A = \{a, b, c, d\}$ and $B = \{b, d, e ,f, g\}$ then

$Jaccard(A, B) = \frac{2}{7}$

We can apply Jaccard similarity by ignoring the rating values.

### Cosine Similarity

Cosine(A, B) =

<img src="cosine.png" width="200">



In [ ]:
def NNCF_based_rating_prediction(u, i, metric):
    r = 0
    sum_sim = 0
    # find movies rated by u
    movies = ratings[ratings["userId"]==u].movieId
    for j in movies:
        sim = calc_sim(i, j, metric)
        key = str(u)+"_"+str(j)
        r += sim*rating_map[key]
        sum_sim += sim
    if sum_sim == 0:
        return 0
    else:
        return r / sum_sim

In [ ]:
# finds the similary of items i and j
def calc_sim(i,j, metric):
    # users who rated item i
    users_rated_i = user_ratings_map[i][0]
    ratings_i = user_ratings_map[i][1]
    # users who rated item j
    users_rated_j = user_ratings_map[j][0]
    ratings_j = user_ratings_map[j][1]

    # Jaccard ignores rating values.
    if metric == "Jaccard":
        intersection_size = len(set(users_rated_i).intersection(users_rated_j))
        union_size = len(set(users_rated_i).union(users_rated_j))
        return intersection_size / union_size
    elif metric == "Cosine":
        inter, ind1, ind2 = np.intersect1d(users_rated_i, users_rated_j, return_indices=True)
        dot = np.dot(ratings_i[ind1], ratings_j[ind2])
        return dot/(np.linalg.norm(ratings_i[ind1])*np.linalg.norm(ratings_j[ind2]))


### Example

In [ ]:
users_i = np.array([1, 3, 6, 9, 12, 15])
ratings_i =  np.array([4, 3, 2, 4, 2, 5])
users_j = np.array([1, 6, 8, 12])
ratings_j = np.array([2, 4, 2, 5])

In [ ]:
inter, ind1, ind2 = np.intersect1d(users_i,users_j,return_indices=True)
print(inter)
print(ind1)
print(ind2)
print(ratings_i[ind1])
print(ratings_j[ind2])
print(np.dot(ratings_i[ind1],ratings_j[ind2]))

## Evaluation of Rating Prediction

How can we measure the performance of a recommender algorithm? This is similar to the evaluation used in machine learning.

- Make a train/test split
- Build the model on the training set
- Make predictions for the ratings in the test set
- Find the mean absolute error (MAE)

For more metrics other then MAE lool at the "Metrics for Regression" section of [this notebook](../../data_science/evaluation.ipynb). For ranking metrics see [this notebook](ranking_evaluation.ipynb)


In [ ]:
ratings = shuffle(ratings)

X_train, X_test = train_test_split(ratings, test_size=100)
train_size = X_train.shape[0]
test_size = X_test.shape[0]
print("Test size:", test_size)
error1 = 0
error2 = 0
error3 = 0
preds = []

avg_rating = X_train.iloc[:,2].mean()
for k in tqdm(range(test_size)):
    u = X_test.iloc[k,0]
    i = X_test.iloc[k,1]
    r = X_test.iloc[k,2]

    error1 += np.abs(r - NNCF_based_rating_prediction(u,i,"Cosine"))
    error2 += np.abs(r - NNCF_based_rating_prediction(u,i,"Jaccard"))
    error3 += np.abs(r - avg_rating)


print("Cosine:", error1/test_size)
print("Jaccard:",error2/test_size)
print("Average:",error3/test_size)


#### Question: How can you explain the difference between using Cosine vs. Jaccard?

## Top-N recommendation Algorithm - Predict and Sort
The task in top-$N$ recommendation is to recommend $N$ items to a user.


Recommend $N$ movies to user $u$
- Predict the ratings of all items which are not watched by $u$
- Sort the predicted ratings
- Recommend the movies with the highest predicted ratings

In [ ]:
def top_N_pred_sort(N, u):
    preds = pd.Series([], dtype='float')
    # find the movies not rated by u
    all_items = set(ratings['movieId'].unique())
    rated_by_user = set(ratings[ratings["userId"]==1].movieId.unique())
    not_rated_by_user = all_items - rated_by_user
    sample_movies = np.random.choice(list(not_rated_by_user), 500)
    for m in tqdm(sample_movies):
        preds[m] = NNCF_based_rating_prediction(u, m, "Jaccard")
    return preds.sort_values(ascending=False)[:N]

In [ ]:
topn = top_N_pred_sort(10, 100)
topn

## Efficiency Issues

There are important inefficiencies in this algorithm:

- The algorithm predicts the rating of all items which are not rated by the user. In the case of millions of items this algorithm is practically infeasible. Numerous techniques have been developed to remedy this problem. Can you suggest a solution?
- In rating prediction, similarity between target item and items rated by the user are calculated. To make a recommendation to another user similarity calculations will be done again. For making recommendations to users in general many similarity calculations will be repeated. A general solution to this problem is to precalculate the similarities between items. Moreover, you don't need to store all similarities, only storing $k$ most similar items to every item will be enough. Size of $k$ can be determined according to the needs.


## Top-N recommendation Algorithm - kNN Map
The task in top-$N$ recommendation is to recommend $N$ items to a user.

- Build a knn-map (a map which stores the $k$ nearest neighbors of each item)

Recommend $N$ movies to user $u$
- Get the neigbors of movies which are watched by $u$ and put them into a list $C$.
- Choose $N$ movies form $C$. There can be different methods here. Most repeated movies in C can be chosen, movies with the highest total similarity (or maximum similarity) can be chosen. These methods will be implemented.
- Recommend the $N$ movies that are chosen.

## Building a knn map
This table will hold the most similar $k$ items for each item. In order to build this table we need to calculate all pairwise similarities which takes $O(n^2)$ time. There is no escape from this $O(n^2)$ time unless you use an approximation algorithm such as LSH (Locality Sensitive Hashing) for nearest neighbor search. Another approach might be to calculate similarites by a matrix multiplication operation and give it to a GPU for accelaration. Yet another approach is discovered by Amazon researchers which can lead to huge speed ups for very sparse matrices which we will look at below.

We will use a heap based priority queue for storing the nearest neighbor. You can look at this [animation](https://www.cs.usfca.edu/~galles/visualization/Heap.html).

In [ ]:
pq =[(10,"a"),(8, "b"), (5, "c"), (3, "d")]
type(pq)
heapq.heapify(pq)
heapq.nsmallest(2,pq)

In [ ]:
movies[:10]

In [ ]:
def build_knn_map(movies, K=30):
    knn_map = {}
    movie_ids = ratings['movieId'].unique()
    print(len(movie_ids))
    for i in tqdm(movie_ids):
        pq = []
        knn_map[i] = pq
        for j in movie_ids:
            if (i == j):
                continue
            sim = calc_sim(i,j,"Jaccard")
            if (len(pq) >= K):
                smallest = pq[0]
                if (sim > smallest[0]):
                    heapq.heappop(pq)
                    heapq.heappush(pq, (sim, j))
            else:
                heapq.heappush(pq, (sim, j))
    return knn_map

In [ ]:
knn_map = build_knn_map(movies)

### Amazon Algorithm

The following is the algorithm used at Amazon. Note that this does not help much for the movielens dataset, however, for very sparse datasets such as the dataset at Amazon, it really helps by skipping many item pairs for which there is no user which rated/bought both items.

Linden, Greg, Brent Smith, and Jeremy York. "Amazon. com recommendations: Item-to-item collaborative filtering." IEEE Internet computing 7.1 (2003): 76-80.


In [ ]:
def build_knn_map_amazon(movies, K=30):
    knn_map = {}
    movie_ids = ratings['movieId'].unique()
    print(len(movie_ids))
    for i in tqdm(movie_ids):
        pq = []
        knn_map[i] = pq
        # find users who rated i
        users = ratings.query("movieId == @i").userId.unique()
        # find items rated by users_i
        movies = ratings.query("userId in @users").movieId.unique()
        # For speed up (this does not exist in the Amazon algorithm)
        movies = np.random.choice(movies, 500)
        for j in movies:
            if (i == j):
                continue
            sim = calc_sim(i,j,"Jaccard")
            if (len(pq) >= K):
                smallest = pq[0]
                if (sim > smallest[0]):
                    heapq.heappop(pq)
                    heapq.heappush(pq, (sim, j))
            else:
                heapq.heappush(pq, (sim, j))
    return knn_map, movies

In [ ]:
knn_map, movies = build_knn_map_amazon(movies)

In [ ]:
def add_sims_and_sort(l):
    li = []
    it = itertools.groupby(l, operator.itemgetter(1))
    for key, subiter in it:
        li.append((key, sum(item[0] for item in subiter)))
    li = sorted(li, key=itemgetter(1), reverse=True)
    return li


In [ ]:
def top_N_knn_map(ratings, N, u):
    C = []
    # find the movies rated by u
    movies_rated = ratings.query("userId == @u").movieId
    for m in movies_rated:
        C = C + knn_map[m]
    return add_sims_and_sort(C)[:N]

In [ ]:
topn = top_N_knn_map(ratings, 10, 100)
topn

In [ ]:
topn = [i[0] for i in topn]
movies[movies.movieId.isin(topn)]

## Evaluation of top-N recommendation

Evaluation of rating prediction is rather easy: find the mean absolute error between rating predictions and true ratings. How can we evaluate the accuracy of a top-N recommendation? There are several techniques which we will look at in more detail later. Below is one common way to evaluate top-N recommendation:

- Randomly sub-sample some portion of positive preferences in order to create a test set $T$. Positive preferences might be 5-star ratings, movies watched more than a certain threshold, or items purchased.
- Put the rest of the preferences into the training set and build model.

- For each preference $(u,i)$ in the test set:
    - Make a top-N recommendation tu user $u$.
    - If the test item i occurs among the top-N items, then we have a hit, otherwise we have a miss.

Hit ratio is then defined as:

$$
Hit Ratio: \frac{\#hits}{|T|}
$$



In [ ]:
N = 100
X_train, X_test = train_test_split(ratings, test_size=1000)
X_test = X_test.query("rating > 4")
train_size = X_train.shape[0]
test_size = X_test.shape[0]
print("Test size:", test_size)
hit_count = 0
for k in range(test_size):
    u = X_test.iloc[k,0]
    i = X_test.iloc[k,1]
    r = X_test.iloc[k,2]
    top_N = top_N_knn_map(X_train, N, u)
    hit_list = [item for item in top_N if item[0] == i]
    if len(hit_list) > 0:
        hit_count +=1
print("Hit Ratio", hit_count/test_size)

### OPTIONAL

The following is a comparison of the running times of similarity calculations performed on the CPU and GPU.

https://github.com/erogluegemen/Performance-and-Scalability-Analysis-of-KNN-Implementations-for-Content-Based-Filtering